#### 原始网页：https://news.qq.com/zt2020/page/feiyan.htm#/
### 1. 数据提取部分

#### 1.1 导入模块

##### 如果使用本地静态资源，需要在头部声明

In [1]:
# # 只需要在顶部声明 CurrentConfig.ONLINE_HOST 即可(针对notebook)
# from pyecharts.globals import CurrentConfig, OnlineHostType
# # OnlineHostType.NOTEBOOK_HOST 默认值为 http://localhost:8888/nbextensions/assets/
# CurrentConfig.ONLINE_HOST = OnlineHostType.NOTEBOOK_HOST

In [2]:
import requests
import json
import time
import datetime
import pandas as pd

In [3]:
today = datetime.datetime.today()
date = f'{today.year}年{today.month}月{today.day}日'
date

'2022年7月13日'

#### 1.2 获取数据

In [4]:
url = "https://api.inews.qq.com/newsqa/v1/query/inner/publish/modules/list?modules=statisGradeCityDetail,diseaseh5Shelf"
response = requests.post(url)
china_datas = response.json()["data"]["diseaseh5Shelf"]["areaTree"][0]["children"]
china_datas[:1]

[{'date': '2022/07/12',
  'today': {'confirm': 19128,
   'confirmCuts': 0,
   'isUpdated': False,
   'tip': '',
   'wzz_add': 0,
   'local_confirm_add': 19128,
   'abroad_confirm_add': 0,
   'dead_add': 96},
  'total': {'adcode': '',
   'highRiskAreaNum': 0,
   'continueDayZeroLocalConfirmAdd': 0,
   'provinceLocalConfirm': 0,
   'continueDayZeroConfirmAdd': 0,
   'nowConfirm': 4079694,
   'wzz': 0,
   'showRate': False,
   'heal': 13742,
   'showHeal': True,
   'mediumRiskAreaNum': 0,
   'confirm': 4101156,
   'dead': 7720,
   'continueDayZeroConfirm': 0,
   'mtime': '2022-07-12 09:56:23'},
  'children': [{'name': '地区待确认',
    'adcode': '',
    'date': '2022/07/12',
    'today': {'confirmCuts': 0,
     'isUpdated': False,
     'wzz_add': '',
     'local_confirm_add': 19128,
     'confirm': 19128},
    'total': {'adcode': '',
     'dead': 7720,
     'heal': 13742,
     'showHeal': True,
     'wzz': 0,
     'showRate': False,
     'provinceLocalConfirm': 0,
     'continueDayZeroLocalCon

#### 1.3 解析提取数据

In [5]:
data_set = []
for data in china_datas:
    data_dict = {
        'province': data['name'], 
        'nowConfirm': data['total']['nowConfirm'], 
        'confirm': data['total']['confirm'], 
        'dead': data['total']['dead'], 
        'heal': data['total']['heal'], 
    } 
    data_set.append(data_dict)

#### 1.4 生成表格对象

In [6]:
df = pd.DataFrame(data_set) 
df

,province,nowConfirm,confirm,dead,heal
0,台湾,4079694,4101156,7720,13742
1,香港,267931,342359,9419,65009
2,澳门,499,642,2,141
3,安徽,262,1446,6,1178
4,广东,184,7626,8,7434
5,上海,169,63431,595,62667
6,天津,152,2033,3,1878
7,吉林,121,40293,5,40167
8,福建,104,3535,1,3430
9,江苏,42,2299,0,2257


#### 1.5 数据永久性保存

In [7]:
df.to_csv('./国内疫情数据.csv', encoding='utf-8-sig')

### 2. 可视化操作部分

#### 2.1 导入模块

In [8]:
from pyecharts import options as opts
from pyecharts.charts import Bar, Line, Pie, Grid, Map

#### 2.2 数据整理

In [9]:
df1 = df.sort_values(by=['nowConfirm'], ascending=False)[:10]
df1

,province,nowConfirm,confirm,dead,heal
0,台湾,4079694,4101156,7720,13742
1,香港,267931,342359,9419,65009
2,澳门,499,642,2,141
3,安徽,262,1446,6,1178
4,广东,184,7626,8,7434
5,上海,169,63431,595,62667
6,天津,152,2033,3,1878
7,吉林,121,40293,5,40167
8,福建,104,3535,1,3430
9,江苏,42,2299,0,2257


#### 2.3 可视化饼图

##### 提示：可视化前，将数据转换为列表

In [10]:
lst = list(zip(df1['province'].values.tolist(),
           df1['nowConfirm'].values.tolist()))
lst

[('台湾', 4079694),
 ('香港', 267931),
 ('澳门', 499),
 ('安徽', 262),
 ('广东', 184),
 ('上海', 169),
 ('天津', 152),
 ('吉林', 121),
 ('福建', 104),
 ('江苏', 42)]

In [11]:
pie = (
    Pie()
    .add("", lst, radius=["10%", "30%"])
    .set_global_opts(
        title_opts=opts.TitleOpts(title=f'{date}_省份占比'),
        legend_opts=opts.LegendOpts(orient="vertical", pos_top="70%", pos_left="70%"))
    .set_series_opts(label_opts=opts.LabelOpts(formatter="{b}:{c}")))
pie.render_notebook()

#### 2.4可视化地图

In [12]:
df3 = df.sort_values(by=['nowConfirm'], ascending=False)[0:]
df3 = df3.drop(df3[df3['nowConfirm'] == 0].index)

lst = list(zip(df3['province'].values.tolist(),
           df3['nowConfirm'].values.tolist()))
map = (
    Map()
    .add("", lst, 'china')
    .set_global_opts(
        title_opts=opts.TitleOpts(title=f'{date}_疫情分布'),
        visualmap_opts=opts.VisualMapOpts(
            max_=1000,
            min_=0,
            is_piecewise=False)
    )
    .set_series_opts(label_opts=opts.LabelOpts(is_show=False))
)
map.render_notebook()
